In [17]:
import json
import pandas as pd
pd.set_option('display.max_rows', None)

In [18]:
with open('/Users/saikrishnakambhampati/Downloads/Project/JSON/amazon_echo.json') as f:
    data = json.load(f)
print(data)

{'ietf-mud:mud': {'mud-version': 1, 'mud-url': 'https://amazonecho.com/amazonecho', 'last-update': '2018-09-16T13:53:04.908+10:00', 'cache-validity': 100, 'is-supported': True, 'systeminfo': 'amazonEcho', 'from-device-policy': {'access-lists': {'access-list': [{'name': 'from-ipv4-amazonecho'}, {'name': 'from-ethernet-amazonecho'}]}}, 'to-device-policy': {'access-lists': {'access-list': [{'name': 'to-ipv4-amazonecho'}]}}}, 'ietf-access-control-list:access-lists': {'acl': [{'name': 'from-ipv4-amazonecho', 'type': 'ipv4-acl-type', 'aces': {'ace': [{'name': 'from-ipv4-amazonecho-0', 'matches': {'ipv4': {'protocol': 6, 'ietf-acldns:dst-dnsname': 'dcape-na.amazon.com'}, 'tcp': {'destination-port': {'operator': 'eq', 'port': 443}, 'ietf-mud:direction-initiated': 'from-device'}}, 'actions': {'forwarding': 'accept'}}, {'name': 'from-ipv4-amazonecho-1', 'matches': {'ipv4': {'protocol': 6, 'ietf-acldns:dst-dnsname': 'softwareupdates.amazon.com'}, 'tcp': {'destination-port': {'operator': 'eq', 'po

In [19]:
def getSourceDestination(data):
    if 'dnsname' in str(data):
        src_net_key = 'ietf-acldns:src-dnsname'
        dest_net_key = 'ietf-acldns:dst-dnsname'
    elif 'network' in str(data):
        src_net_key = 'source-ipv4-network'
        dest_net_key = 'destination-ipv4-network'
    else:
        src_net_key = '*'
        dest_net_key = '*'
    return data.get(src_net_key, '*'), data.get(dest_net_key, '*'), data.get('protocol', '*')
    

In [27]:
access_json = data['ietf-access-control-list:access-lists']['acl']
final_list = []
for acl in access_json:
    aces = acl['aces']['ace']
    for ace in aces:
        final_row = {
            'sEth': '*',
            'dEth': '*',
            'typEth': '*',
        }
        if 'ethertype' in str(ace['matches']):
            final_row['sEth'] = ace['matches']['eth'].get('source-mac-address', '*')
            final_row['dEth'] = ace['matches']['eth'].get('destination-mac-address', '*')
            final_row['typEth'] = ace['matches']['eth'].get('ethertype', '*')
        if 'ipv4' in str(ace['matches']):
            source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
            final_row['Source'] = source
            final_row['Destination'] = dest
            final_row['proto'] = proto
        elif 'ipv6' in str(ace['matches']):
            source, dest, proto = getSourceDestination(ace['matches']['ipv6'])
            final_row['Source'] = source
            final_row['Destination'] = dest
            final_row['proto'] = proto
        else:
            final_row['Source'] = '*'
            final_row['Destination'] = '*'
            final_row['proto'] = '*'
        if 'ipv4' in str(ace['matches']):
            source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
            if proto in [1,6,17] :
                final_row['typEth']='0x800'
        if 'icmp' in str(ace['matches']):
            final_row['type'] = ace['matches']['icmp'].get('type', '*') 
            final_row['code'] = ace['matches']['icmp'].get('code', '*') 
        else:
            final_row['type'] = '*' 
            final_row['code'] = '*'   
        if 'source-port' in str(ace['matches']):
            final_row['sPort'] = ace['matches']['udp']['source-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['source-port'].get('port', '*')
        else:
            final_row['sPort'] = '*'
        if 'destination-port' in str(ace['matches']):
            final_row['dPort'] = ace['matches']['udp']['destination-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['destination-port'].get('port', '*')
        else:
            final_row['dPort'] = '*'
        final_row['priority'] = '*'
        final_row['action'] = 'forward' if ace['actions'].get('forwarding', '') == 'accept' else '*'
        if final_row['Source'] == '*' and final_row['Destination'] != '*':
            final_row['sEth'] = '9e:8d:de:80:29:28'  
        if final_row['Destination'] == '*' and final_row['Source'] != '*':
            final_row['dEth'] = '9e:8d:de:80:29:28'     
        final_list.append(final_row)
# print(json.dumps(final_list))
df = pd.DataFrame(final_list)
df

,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,0x800,*,dcape-na.amazon.com,6,*,*,*,443,*,forward
1,9e:8d:de:80:29:28,*,0x800,*,softwareupdates.amazon.com,6,*,*,*,443,*,forward
2,9e:8d:de:80:29:28,*,0x800,*,3.north-america.pool.ntp.org,17,*,*,*,123,*,forward
3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,*,*,forward
4,9e:8d:de:80:29:28,*,0x800,*,239.255.255.250/32,17,*,*,*,1900,*,forward
5,9e:8d:de:80:29:28,ff:ff:ff:ff:ff:ff,0x800,*,255.255.255.255/32,17,*,*,*,67,*,forward
6,9e:8d:de:80:29:28,*,0x800,*,ntp-g7g.amazon.com,17,*,*,*,123,*,forward
7,9e:8d:de:80:29:28,*,0x800,*,todo-ta-g7g.amazon.com,6,*,*,*,443,*,forward
8,*,*,0x800,*,*,1,8,0,*,*,*,forward
9,9e:8d:de:80:29:28,*,0x800,*,www.example.org,6,*,*,*,80,*,forward


In [117]:
df.to_csv('/Users/saikrishnakambhampati/Downloads/Project/Data/withingssleepsensor.csv')